In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

Using TensorFlow backend.


In [2]:
def HappyModel(input_shape, output_shape, hpar):
    X_input = Input(input_shape, name = 'input0') # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X = X_input
    L = len(hpar['layer_conv_filter'])
    for i in range(0, L):
        conv_ksize = tuple(hpar['layer_conv_filter'][i][0:2])
        conv_filters = hpar['layer_conv_filter'][i][2]
        conv_stride = tuple(hpar['layer_conv_stride'][i])
        conv_pad = hpar['layer_conv_padding'][i]
        pool_ksize = tuple(hpar['layer_pool_filter'][i])
        pool_stride = tuple(hpar['layer_pool_stride'][i])
        pool_pad = hpar['layer_pool_padding'][i]
        X = Conv2D(conv_filters, conv_ksize, strides = conv_stride, padding = conv_pad, name = 'conv%d'%i)(X) # CONV -> BN -> RELU Block applied to X
        X = BatchNormalization(axis = 3, name = 'bn%d'%i)(X) # CONV -> BN -> RELU Block applied to X
        X = Activation('relu', name = 'activation%d'%i)(X) # CONV -> BN -> RELU Block applied to X
        X = MaxPooling2D(pool_size = pool_ksize, strides = pool_stride, padding = pool_pad, name='max_pool%d'%i)(X) # MAXPOOL
    X = Flatten(name = 'flatten')(X) # FLATTEN X (means convert it to a vector)
    Lf = len(hpar['hlayer_fully_connected'])
    for i in range(0, Lf):
        num_units = hpar['hlayer_fully_connected'][i]
        X = Dense(num_units, activation='relu', name='fc%d'%i)(X) # FULLYCONNECTED
    X = Dense(output_shape, activation='sigmoid', name='fc%d'%Lf)(X) # FULLYCONNECTED
    model = Model(inputs = X_input, outputs = X, name='HappyModel') # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    return model

In [3]:
def load_HAPPY_data(dirpath = './datasets/coursera_dnn4_week2_Keras Tutorial - The Happy House/datasets/'):
    import h5py
    train_dataset = h5py.File(dirpath+'train_happy.h5', "r")
    X_train_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    Y_train_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels
    test_dataset = h5py.File(dirpath+'test_happy.h5', "r")
    X_test_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    Y_test_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels
    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    Y_train_orig = Y_train_orig.reshape((1, Y_train_orig.shape[0]))
    Y_test_orig = Y_test_orig.reshape((1, Y_test_orig.shape[0]))
    #-------------------------------------------------------
    X_train = X_train_orig/255. # Normalize image vectors
    X_test = X_test_orig/255.
    Y_train = Y_train_orig.T # Reshape
    Y_test = Y_test_orig.T
    return X_train, Y_train, X_test, Y_test, classes
if True:
    X_train, Y_train, X_test, Y_test, classes = load_HAPPY_data()
    print ("number of training examples = " + str(X_train.shape[0]))
    print ("number of test examples = " + str(X_test.shape[0]))
    print ("X_train shape: " + str(X_train.shape))
    print ("Y_train shape: " + str(Y_train.shape))
    print ("X_test shape: " + str(X_test.shape))
    print ("Y_test shape: " + str(Y_test.shape))
    #-------------------------------------------------------
    hpar = {}
    hpar['layer_conv_filter'] = [[7,7,32]] # [height, width, channel]
    hpar['layer_conv_stride'] = [[1,1]] # [height, width]
    hpar['layer_conv_padding'] = ['SAME']
    hpar['layer_pool_filter'] = [[2,2]] # [height, width]
    hpar['layer_pool_stride'] = [[2,2]] # [height, width]
    hpar['layer_pool_padding'] = ['VALID']
    hpar['hlayer_fully_connected'] = []
    #-------------------------------------------------------
    model = HappyModel((64,64,3), 1, hpar)
    model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
    model.summary()
    model.fit(x = X_train, y = Y_train, epochs = 40, batch_size = 16)
    preds = model.evaluate(x = X_test, y = Y_test)
    print()
    print ("Loss = " + str(preds[0]))
    print ("Test Accuracy = " + str(preds[1]))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input0 (InputLayer)          (None, 64, 64, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 32)        4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 64, 64, 32)        128       
_________________________________________________________________
activation0 (Activation)     (None, 64, 64, 32)        0         
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 32, 32, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None

In [4]:
"""
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_4 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
zero_padding2d_4 (ZeroPaddin (None, 70, 70, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 32)        4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 64, 64, 32)        128       
_________________________________________________________________
activation_4 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 32768)             0         
_________________________________________________________________
fc (Dense)                   (None, 1)                 32769     
=================================================================
Total params: 37,633
Trainable params: 37,569
Non-trainable params: 64
"""
print()